In [12]:
import numpy as np
import cv2
import random
from ultralytics import YOLO
from sort.sort import Sort
import imageio
import os
import pygifsicle 


In [ ]:
def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)
# plot_one_box(bboxes, overlayImage, label=label, color=color, line_thickness=line_thickness, bottom_label=bottom_label)
def plot_one_box(x, img, color=None, label=None, line_thickness=None, bottom_label=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1# line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    
    tf = max(tl - 1, 1)# font thickness
    if label:
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c4 = c1[0] + t_size[0], c1[1] - t_size[1] - 3# filled
        cv2.rectangle(img, c1, c4, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    if bottom_label:
        a_size = cv2.getTextSize(bottom_label, 0, fontScale=tl / 4, thickness=tf)[0]
        c3 = c1[0] + a_size[0], c2[1] + a_size[1] + 3
        cv2.rectangle(img, (c1[0], c2[1]), c3, color, -1, cv2.LINE_AA)
        cv2.putText(img, bottom_label, (c1[0], c2[1] + 12), 0, tl / 4, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def plot_text(up_text,img,c1=(546,40),c2=(550,50)):
    tl = 3
    tf = max(tl - 1, 1)# font thickness
    up_text_size = cv2.getTextSize(up_text, 0, fontScale=tl / 3, thickness=tf)[0]
    c4 = c1[0] + up_text_size[0], c1[1] - up_text_size[1] - 3# filled
    cv2.rectangle(img, c1, c4, (0,0,255), -1, cv2.LINE_AA)
    cv2.putText(img, up_text, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    
    

In [ ]:
def resize_img(im, target_width = 640):
    h,w,_  = im.shape
    target_height = int(h / w * target_width)
    im = cv2.resize(im , (target_width , target_height), interpolation = cv2.INTER_AREA)  
    return im,target_height,target_width

In [ ]:
def xyxy_xywh(x1,y1,x2,y2):
    w = abs(x2-x1)
    h = abs(y2-y1)
    x,y = x1+w/2,y1+h/2
    return x,y,w,h
def xywh_xyxy(x,y,w,h):
    x1 = x-w/2
    y1 = y-h/2
    x2 = x+w/2
    y2 = y+h/2
    return int(x1),int(y1),int(x2),int(y2)

def scale_con(x,y,w,h,old,new):
    # old : tuple(height,width)
    x = int(x* old[0]/new[0])
    y = int(y*old[1]/new[1])
    h = int(h*old[0]/new[0])
    w = int(w*old[1]/new[1])
    return x,y,w,h 
def isInPolygon(PointF, polygon):
    i = 0
    c = False
    j = len(polygon) - 1
    for i in range(len(polygon)):        
        if (((polygon[i][1] > PointF[1]) != (polygon[j][1] > PointF[1])) and 
        (PointF[0] < (polygon[j][0] - polygon[i][0]) * (PointF[1] - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0])):
            c = not(c)
        j = i
    return c

In [ ]:
# load model 
model = YOLO("./runs/detect/train/weights/best.pt")

In [ ]:
# Tracker Initialization
sort_tracker = Sort(max_age=20,
                   min_hits=5,
                   iou_threshold=0.3)

In [ ]:
# video Visualization
# press y to quit the video
cap = cv2.VideoCapture("./road_vehicle.mp4")
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
# Video writer

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('processed_road_vehicle_counted.mp4', fourcc, fps, (frame_width, frame_height))


print(frame_width,frame_height)

# polygons for counting upside and downside vehicles
up_polylines = np.array([[390,350],[319,382],[585,428],[599,368]])
down_polylines = np.array([[694,444],[720,534],[1105,465],[991,404]])
up_vehicles = []
down_vehicles = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        img,res_height,res_width = resize_img(frame)
        
        cv2.polylines(frame, [up_polylines], True, (0,0,255), 2)
        cv2.polylines(frame, [down_polylines], True, (0,0,255), 2)
        plot_text("UP Vehicles : {}".format(len(up_vehicles)),frame,(546,40),(550,50))
        plot_text("Down Vehicles : {}".format(len(down_vehicles)),frame,(546,75),(600,50))
        result = model.predict(img)[0]
        
        
        
        
        bboxes = np.array(result.boxes.xywh)
        mapped_bboxes = [[0]]*len(bboxes)
        
        
        
        for ind,bbox in enumerate(bboxes):
            x,y,w,h = scale_con(bbox[0],bbox[1],bbox[2],bbox[3],(frame_height,frame_width),(res_height,res_width))
            x1,y1,x2,y2 = xywh_xyxy(x,y,w,h)
            
            mapped_bboxes[ind] = [x1,y1,x2,y2]
        # tracker updations
        outputs = sort_tracker.update(np.array(mapped_bboxes))
        
        for box in outputs:
            x1,y1,x2,y2,tid, = box
            plot_one_box(x=(x1,y1,x2,y2),img=frame,color=compute_color_for_id(int(tid))
                         ,label=str(int(tid)),line_thickness=1)
            x,y,w,h = xyxy_xywh(x1,y1,x2,y2)
            
            # check bbox belong to wich polygon to add counter in vehicle passing
            
            if isInPolygon((x,y),up_polylines):
                if tid not in up_vehicles:
                    up_vehicles.append(tid)
            elif isInPolygon((x,y),down_polylines):
                if tid not in down_vehicles:
                    down_vehicles.append(tid)   
            
            
        output.write(frame)
#         cv2.imshow('Frame',frame)
#         k =cv2.waitKey(0) & 0xFF
#         #press y to exit
#         if k==121:
#             break
#         else:
#             pass
    else:
        break
cap.release()
# cv2.destroyAllWindows()
output.release()

In [8]:
# writing processed and unprocessed video file side by side
cap1 = cv2.VideoCapture("./road_vehicle.mp4")
cap2 = cv2.VideoCapture("./processed_road_vehicle_counted.mp4")
frame_width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
target_width = frame_width//2
target_height = frame_height//2
fps = cap1.get(cv2.CAP_PROP_FPS)

# Video writer

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('joined_video.mp4', fourcc, fps, (frame_width, target_height))


# target_frames = 800
print(frame_width,frame_height,fps)
print(target_width,target_height,fps)
while(cap1.isOpened()):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if ret1 == True and ret2 == True :
        res_frame1 = cv2.resize(frame1 , (target_width , target_height), interpolation = cv2.INTER_AREA)  
        res_frame2 = cv2.resize(frame2 , (target_width , target_height), interpolation = cv2.INTER_AREA) 
        concat_frame = np.concatenate((res_frame1, res_frame2), axis=1)
        
        output.write(concat_frame)
#         cv2.imshow('Frame',concat_frame)
#         k =cv2.waitKey(0) & 0xFF
#         #press y to exit
#         if k==121:
#             break
#         else:
#             pass
    else:
        break
cap1.release()
cap2.release()
cv2.destroyAllWindows()
output.release()

1280 720 25.0
640 360 25.0


In [5]:
import imageio
import moviepy.editor as mp
from moviepy.editor import VideoFileClip

In [8]:


def convert_video_to_gif(input_video_path, output_gif_path, fps=25):
    video_clip = mp.VideoFileClip(input_video_path)
    video_clip.write_gif(output_gif_path, fps=fps)


In [10]:
video_file = "./joined_video.mp4"
output_gif_file = "./joined_video.gif"
convert_video_to_gif(video_file, output_gif_file)

MoviePy - Building file ./joined_video.gif with imageio.


In [20]:

def convert_half_video_to_gif(input_video_path, output_gif_path):
    # Load the video clip
    video_clip = VideoFileClip(input_video_path)

    # Get the duration of the video
    total_duration = video_clip.duration

    # Set the end time for the first half
    end_time = total_duration / 2
#     print(end_time,type(end_time))

    # Trim the video to the first half
    trimmed_clip = video_clip.subclip(0, 10.0)

    # Write the trimmed video as a GIF
    trimmed_clip.write_gif(output_gif_path)

video_file = "./joined_video.mp4"
output_gif_file = "./clipped_joined_video.gif"
convert_half_video_to_gif(video_file, output_gif_file)



                                                         

t:   7%|▋         | 30/459 [5:23:11<02:10,  3.29it/s, now=None]

MoviePy - Building file ./clipped_joined_video.gif with imageio.
